In [ ]:
import numpy as np
import os
import random
import warnings

from modules import helper
from modules import dataset_selecter as ds
from modules import modelCreator

from datetime import datetime


import tensorflow as tf
from tensorflow.python.eager import context
from tensorflow.python.framework import ops
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import StratifiedKFold

In [ ]:
nrFolds= 5
patience= 70
seed_value= 56
batchSize= 50
useEmbed= False # for now removed
calcOri= True # calc the original model
calcSFA= True # calc the original model
doSymbolify= True # symbolify the data (if the data is maybe already symbolified)
useSaves= False # dont calculate already calcluated model runs
useShapeSaves= False
multiVariant= False # not working right now
skipDebugSaves= False # reduces the saved amount of data
dropeOutRate= 0.3
takeAvg= True # False if one specific Transformer layer should be used
heatLayer= -1 # The Layer which should be used if takeAvg is False

dataset= 'ucr' #
takename= True #If True, take UTC names rather than numbers
limit= 500 # max sequence size for a dataset
numEpochs= 500


#number of symbols
symbolCount= 6

numOfAttentionLayers=2

# dataset to use from UCR
number="Meat"
strategy='uniform'

header= 8
# number of coefs. [0] is the minimal coef size, weile [1] is a factor on how to handle sequences < [0]. [0,0] means SAX is getting used.
ncoef= [0, 0] #[256, 1]  [128, 2], [64, 4], [0,0]

dmodel=16
dff= 8


In [ ]:
os.environ['PYTHONHASHSEED']=str(seed_value)# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)# 3. Set `numpy` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)
np.random.RandomState(seed_value)

np.random.seed(seed_value)

context.set_global_seed(seed_value)
ops.get_default_graph().seed = seed_value

#pip install tensorflow-determinism needed
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(seed_value)

#save some variables for later
valuesA = helper.getMapValues(symbolCount)
kf = StratifiedKFold(nrFolds, shuffle=True, random_state=seed_value)
earlystop = EarlyStopping(monitor= 'val_loss', min_delta=0 , patience=patience, verbose=0, mode='auto',restore_best_weights=True)
fold = 0

#init gpu
physical_devices = tf.config.list_physical_devices('GPU') 
for gpu_instance in physical_devices: 
    tf.config.experimental.set_memory_growth(gpu_instance, True)

In [ ]:
X_train, X_test, y_train, y_test, y_trainy, y_testy, seqSize, dataName, num_of_classes, number = ds.datasetSelector(dataset, seed_value, number, takeName=takename)

In [ ]:
def fillOutDicWithNNOutFull(abstractionString, configString, fullResults, inputDict):
    if abstractionString not in fullResults.keys():
        fullResults[abstractionString] = dict()
    if configString not in fullResults[abstractionString].keys():
        fullResults[abstractionString][configString] = []
    fullResults[abstractionString][configString].append(inputDict)



def fillOutDicWithNNOut(abstractionString, configString, fullResults, outData):
    inputDict = helper.fillOutDicWithNNOutSmall(outData)
    fillOutDicWithNNOutFull(abstractionString, configString, fullResults, inputDict)

def printTime():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)

In [ ]:
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")
print('Dataname:')
print(dataName)
printTime()
warnings.filterwarnings('ignore')   

fullResults = dict()

#limit the lenght of the data
toLong = False
dontRun = False
if seqSize > limit:
    if calcSFA and ncoef[0] != 0:
        toLong = True
        print('TO LONGE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print('Only calculating SFA!')
    else:
        fullResults["Error"] = "dataset " + dataName + " to big: " + str(seqSize)
        print('TO LONGE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        print("dataset " + dataName + " to big: " + str(seqSize))
        dontRun = True

if not dontRun:
    # k fold train loop
    for train, test in kf.split(X_train, y_trainy):
        fold+=1    

        #preprocess data
        x_train1 = X_train[train]
        x_val = X_train[test]
        y_train1 = y_train[train]
        y_trainy1 = y_trainy[train]
        y_val = y_train[test]
        
        x_train1, x_val, x_test, y_train1, y_val, y_test, X_train_ori, X_val_ori, X_test_ori, y_trainy1, y_testy = modelCreator.preprocessData(x_train1, x_val, X_test, y_train1, y_val, y_test, y_trainy1, y_testy, fold, symbolCount, dataName, useEmbed = useEmbed, useSaves = useShapeSaves, doSymbolify = doSymbolify, multiVariant=multiVariant, strategy=strategy)

        # calc original model
        abstractionString = "Original"
        if(calcOri and not toLong):
            print('############################### calc ori')
            outOri = modelCreator.doAbstractedTraining(X_train_ori, X_val_ori, X_test_ori, y_train1, y_val, y_testy, batchSize, seed_value, num_of_classes, dataName, fold, symbolCount, numEpochs, numOfAttentionLayers, dmodel, header, dff, skipDebugSaves=skipDebugSaves, useEmbed = useEmbed, earlystop = earlystop, useSaves=useSaves, 
            abstractionType=abstractionString, rate=dropeOutRate)
            fillOutDicWithNNOut(abstractionString, "results", fullResults, outOri)
        printTime()

        # calc SAX or SFA model
        if (not calcSFA) or (ncoef[0] == 0):
            abstractionString = "SAX"
            outSax = modelCreator.doAbstractedTraining(x_train1, x_val, x_test, y_train1, y_val, y_testy, batchSize, seed_value, num_of_classes, dataName, fold, symbolCount, numEpochs, numOfAttentionLayers, dmodel, header, dff, skipDebugSaves=skipDebugSaves, useEmbed = useEmbed, earlystop = earlystop, useSaves=useSaves, 
                abstractionType=abstractionString, rate=dropeOutRate)
            printTime()
            fillOutDicWithNNOut(abstractionString, "results", fullResults, outSax)   
        else:
            limitS = seqSize/ncoef[1]
            if ncoef[0] > limitS:
                ncoefI = int(limitS)
            else:
                ncoefI = ncoef[0]


            x_train1 = X_train[train]
            x_val = X_train[test]
            y_train1 = y_train[train]
            y_trainy1 = y_trainy[train]
            y_val = y_train[test]

            x_train1, x_val, x_test, y_train1, y_val, y_test, X_train_ori, X_val_ori, X_test_ori, y_trainy1, y_testy = modelCreator.preprocessData(x_train1, x_val, X_test, y_train1, y_val, y_test, y_trainy1, y_testy, fold, symbolCount, dataName, useEmbed = useEmbed, useSaves = useShapeSaves, doSymbolify = doSymbolify, multiVariant=multiVariant, doSFA=True, ncoef=ncoefI, strategy=strategy)

            abstractionString = "SFA" + str(ncoef[1])
            outSax = modelCreator.doAbstractedTraining(x_train1, x_val, x_test, y_train1, y_val, y_testy, batchSize, seed_value, num_of_classes, dataName, fold, symbolCount, numEpochs, numOfAttentionLayers, dmodel, header, dff, skipDebugSaves=skipDebugSaves, useEmbed = useEmbed, earlystop = earlystop, useSaves=useSaves, 
                abstractionType=abstractionString, rate=dropeOutRate, ncoef=ncoef[0])
            printTime()
            fillOutDicWithNNOut(abstractionString, "results", fullResults, outSax)
            fullResults['ncoef']= ncoef

        predictions = np.argmax(y_train1,axis=1) +1

        print("finished fold: " + str(fold))
        printTime()   
        break
             
    print("Done done")
    saveName = modelCreator.getWeightName(dataName, number, symbolCount, numOfAttentionLayers, "results", header, dmodel=dmodel, dff=dff,doDetails=True, learning = False, results = True, posthoc= str(ncoef[0]) + " -st" + strategy[0:1])
    print(saveName)
    helper.save_obj(fullResults, str(saveName))


    printTime()
